# MLSZ Adatbank szkennelése

A program célja, hogy az MLSZ adatbankban szereplő utánpótlás korú játékosok adatait naprakészen tartsuk. Valamint a kiemelkedő értékekkel rendelkező fiatalok ne vesszenek el a rendszerben és a személyes megtekintéseket optimalizálni tudjuk.



In [65]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

## Alapadatok beolvasása

#### Példa link
https://adatbank.mlsz.hu/player/437473.html

A teljes oldal tartalom letöltése

In [66]:
link = 'https://adatbank.mlsz.hu/player/437473.html'
html = urlopen(link)
bs = BeautifulSoup(html, "html.parser")
bs

<!DOCTYPE html>

<html lang="hu"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport">
<link href="https://adatbank.mlsz.hu/meccs-center/img/mlsz-logo-kicsi.png" rel="shortcut icon"/>
<title>TÓTH BORISZ - DVTK - MLSZ adatbank</title>
<meta content="" name="description"/>
<script>
		var Protocol = 'https://';
	</script>
<script async="" src="https://www.google-analytics.com/ga.js" type="text/javascript"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="https://code.jquery.com/ui/1.12.1/jquery-ui.min.js"></script>
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
<script src="https://adatbank.mlsz.hu/js/script.js?t1589904358.6403"></script>
<script src="https://ada1bank.mlsz.hu/meccs-center/js/datepicker.js"></script>
<script src="https://ada1bank.mlsz.hu/meccs-center/js/lightslider.js

### Hasznos adatok kinyerése az őskáoszból

Név, születési idő, kor, aktuális csapat

In [67]:
basedata = bs.findAll('td', {'class': 'datas'})
name = basedata[0].get_text()
dateofbirth = basedata[1].get_text()
age = basedata[2].get_text()
actualclub = basedata[3].get_text()

In [68]:
print(name)
print(dateofbirth)
print(age)
print(actualclub)

TÓTH BORISZ
2002. 07. 07.
17

DVTK (Labdarúgás) 


In [69]:
seasondata = bs.findAll('th')
seasondata

[<th class="szezon_header"><div class="all_matches_table_season">
                                         2019/2020 - DVTK</div></th>,
 <th>Ö<span class="under700">sszes</span></th>,
 <th>K<span class="under700">ezdő</span></th>,
 <th>Cs<span class="under700">ere</span></th>,
 <th>Kispad</th>,
 <th>G<span class="under700">ól</span></th>,
 <th>Öngól</th>,
 <th>S<span class="under700">árga</span></th>,
 <th>P<span class="under700">iros</span></th>,
 <th></th>,
 <th></th>,
 <th>19</th>,
 <th>13</th>,
 <th>1</th>,
 <th>5</th>,
 <th>2</th>,
 <th>0</th>,
 <th>2</th>,
 <th>0</th>,
 <th class="player_drop_icon" onclick="showLeaguesOnPlayer(this);"></th>,
 <th class="szezon_header"><div class="all_matches_table_season">
                                         2018/2019 - DVTK</div></th>,
 <th>Ö<span class="under700">sszes</span></th>,
 <th>K<span class="under700">ezdő</span></th>,
 <th>Cs<span class="under700">ere</span></th>,
 <th>Kispad</th>,
 <th>G<span class="under700">ól</span></th>,
 <t

### Az aktuális szezon számainak kiolvasása

Melyik szezon, Összes meccs, Kezdő, Csere, Kispad, Gól, Öngól, Sárga- és Piros Lap. 

In [70]:
WhichSeason = seasondata[0].get_text()
AllMatch = seasondata[11].get_text()
Starter = seasondata[12].get_text()
Sub = seasondata[13].get_text()
Bench = seasondata[14].get_text()
Gol = seasondata[15].get_text()
OwnGol = seasondata[16].get_text()
YellowCard = seasondata[17].get_text()
RedCard = seasondata[18].get_text()

print(WhichSeason)
print(AllMatch)
print(Starter)
print(Sub)
print(Bench)
print(Gol)
print(OwnGol)
print(YellowCard)
print(RedCard)


                                        2019/2020 - DVTK
19
13
1
5
2
0
2
0


### Üres Adatlapok kezelése

Abban az esetben ha az adott MLSZ adatlap nem tartalmaz játékos adatokat.
Pl.: https://adatbank.mlsz.hu/player/595055.html

In [71]:
link2 = 'https://adatbank.mlsz.hu/player/595055.html'
html2 = urlopen(link2)
bs2 = BeautifulSoup(html2, "html.parser")
bs2

<!DOCTYPE html>

<html lang="hu"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport">
<link href="https://adatbank.mlsz.hu/meccs-center/img/mlsz-logo-kicsi.png" rel="shortcut icon"/>
<title> -  - MLSZ adatbank</title>
<meta content="" name="description"/>
<script>
		var Protocol = 'https://';
	</script>
<script async="" src="https://www.google-analytics.com/ga.js" type="text/javascript"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="https://code.jquery.com/ui/1.12.1/jquery-ui.min.js"></script>
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
<script src="https://adatbank.mlsz.hu/js/script.js?t1589904413.9406"></script>
<script src="https://ada1bank.mlsz.hu/meccs-center/js/datepicker.js"></script>
<script src="https://ada1bank.mlsz.hu/meccs-center/js/lightslider.js"></script>
<!-

In [72]:
basedata2 = bs2.findAll('td', {'class': 'datas'})
name2 = basedata2[0].get_text()
name2

''

Tehát ha üres a név mező akkor nem szükséges további adatokat kinyerni az oldalról

### Az adathalmaz rendszerezése és exportálása fájlba

In [74]:
PlayerDataSheet = []
PlayerDataSheet.append(link)
PlayerDataSheet.append(name)
PlayerDataSheet.append(dateofbirth)
PlayerDataSheet.append(age)
PlayerDataSheet.append(actualclub)
PlayerDataSheet.append(WhichSeason)
PlayerDataSheet.append(AllMatch)
PlayerDataSheet.append(Starter)
PlayerDataSheet.append(Sub)
PlayerDataSheet.append(Bench)
PlayerDataSheet.append(Gol)
PlayerDataSheet.append(OwnGol)
PlayerDataSheet.append(YellowCard)
PlayerDataSheet.append(RedCard)

PlayerDataSheet

['https://adatbank.mlsz.hu/player/437473.html',
 'TÓTH BORISZ',
 '2002. 07. 07.',
 '17',
 '\nDVTK (Labdarúgás) ',
 '\n                                        2019/2020 - DVTK',
 '19',
 '13',
 '1',
 '5',
 '2',
 '0',
 '2',
 '0']